# MGSC 662 Final Project Model

In [3]:
import pandas as pd
import numpy as np

import gurobipy as gb
from gurobipy import *

In [4]:
df = pd.read_csv("C:/Users/Tony/Downloads/boutique_data (1).csv")

## Data Pre-processing

In [5]:
cols_to_dummies =['Gender','Comm. Category SubGroup']

In [6]:
dummies = pd.get_dummies(df[cols_to_dummies])
df_with_dummies = pd.concat([df, dummies], axis=1)

In [7]:
df_with_dummies

,Style-Fabric-Color,Gender,Birth Season,Comm. Category SubGroup,new,vip,revenue,Gender_Female,Gender_Male,Gender_Unisex,Comm. Category SubGroup_BAGS,Comm. Category SubGroup_BELT,Comm. Category SubGroup_FASHION JEWELRY,Comm. Category SubGroup_HOME,Comm. Category SubGroup_LICENSES,Comm. Category SubGroup_RTW,Comm. Category SubGroup_SHOES,Comm. Category SubGroup_SILK & OTHER ACC,Comm. Category SubGroup_SLG
0,2 PEZZI BICCH. LONGDRINK CONF.REG.MEDUSA LUMIE...,Unisex,-missing description-,HOME,20,0,13285.230610,0,0,1,0,0,0,1,0,0,0,0,0
1,2 PEZZI BICCHIERE LONGDRINK CONF. REG. MEDUSA ...,Unisex,Not Seasonal 2016,HOME,2,0,418.262695,0,0,1,0,0,0,1,0,0,0,0,0
2,2 PEZZI BICCHIERE WHISKY CONF. REG. MEDUSA DOR...,Unisex,-missing description-,HOME,11,1,8748.707520,0,0,1,0,0,0,1,0,0,0,0,0
3,2 PEZZI BICCHIERE WHISKY CONF. REG. MEDUSA LUM...,Unisex,-missing description-,HOME,86,0,32723.878750,0,0,1,0,0,0,1,0,0,0,0,0
4,ABB. SPORTIVO - FELPA CON CAPPUCCIO TRIBUTE-Bl...,Male,Spring / Summer 2018,RTW,45,5,222176.896200,0,1,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5839,ZAINO TESS.STAMP.CAMOUFLAGE+VITELLO-Multicolor...,Male,Spring / Summer 2018,BAGS,1,0,1110.768555,0,1,0,1,0,0,0,0,0,0,0,0
5840,ZAINO VIT.C/GRANA STAMPA LETTERING-Multicolor-...,Male,Spring / Summer 2019,BAGS,3,0,3479.013428,0,1,0,1,0,0,0,0,0,0,0,0
5841,ZAINO VIT.SAFFIANO STAMPA NEWSPAPER+ VITELLO-W...,Male,Spring / Summer 2019,BAGS,0,0,4634.877808,0,1,0,1,0,0,0,0,0,0,0,0
5842,ZAINO VITELLO NAPPATO LAV.CAPITONNE'-BLACK-TRI...,Male,Fall / Winter 2018-2019,BAGS,2,0,9036.813110,0,1,0,1,0,0,0,0,0,0,0,0


In [8]:
new = df['new'].tolist()
vip=df['vip'].tolist()
revenue=df['revenue'].tolist()
style = df['Style-Fabric-Color'].tolist()

In [9]:
columns_to_include = ['Gender_Male',
                      'Gender_Female',
                      'Gender_Unisex']

In [10]:
Gender = df_with_dummies[columns_to_include].values.tolist()

In [11]:
new = df['new'].tolist()
vip=df['vip'].tolist()
revenue=df['revenue'].tolist()
style = df['Style-Fabric-Color'].tolist()

In [12]:
columns_to_include = ['Comm. Category SubGroup_BAGS',
                      'Comm. Category SubGroup_BELT'
                      ,'Comm. Category SubGroup_FASHION JEWELRY',
                      'Comm. Category SubGroup_HOME'
                     ,'Comm. Category SubGroup_LICENSES',
                      'Comm. Category SubGroup_RTW',
                      'Comm. Category SubGroup_SHOES',
                     'Comm. Category SubGroup_SILK & OTHER ACC',
                      'Comm. Category SubGroup_SLG']

In [13]:
Category = df_with_dummies[columns_to_include].values.tolist()

## Display Position Formulation

In [14]:
display_case = [
    range(0,2),
    range(2,5),
    range(5,10),
    range(10,15),
    range(15,20),
    range(20,25),
    range(25,35),
    range(35,45),
    range(45,55),
    range(55,65),
    range(65,75),
    range(75,79),
    range(79,89)
]
shopwindow = range(0,4)
bags_and_accessories = range(4,24)
women_clothings = range(24,44)
men_clothings = range(44,64)
belt = range(64,74)
jewery = range(74,78)
SLG = range(78,88)

In [15]:
birthseason = df_with_dummies["Birth Season"].apply(lambda x: 1 if x == "Spring / Summer 2019" else 0).tolist()

## Problem Formulation - VIP Priority

In [16]:
num = len(df_with_dummies)

model = gb.Model("Question1")
model.setParam('MIPGap', 0.000000000001)

# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

X = model.addVars(88,num,
                  vtype=GRB.BINARY, 
                  name=['location'+str(i)+j for i in range(88) for j in style])

Y = model.addVars(88,
                  vtype=GRB.BINARY, 
                  name=['location discount'+str(i) for i in range(88)])

ne1 = quicksum(new[j]*X[i,j] for i in range(4,88) for j in range(num))
ne2=quicksum(new[j]*X[i,j] for i in range(0,4) for j in range(num))*1.1

vi1 = quicksum(vip[j]*X[i,j] for i in range(88) for j in range(num))

rev1 = quicksum(revenue[j]*X[i,j] for i in range(4,88) for j in range(num))
rev2= quicksum(revenue[j]*X[i,j] for i in range(0,4) for j in range(num))*1.1


model.setObjectiveN(-(ne1+ne2), index=0, priority=0)
model.setObjectiveN(-rev1-rev2, index=1, priority=1)
model.setObjectiveN(-vi1, index=2, priority=2)

# 1 good for every postiion
for i in range(88):
    model.addConstr(quicksum(X[i,j] for j in range(num))==1)


#one product only appear once
for j in range(num):
    model.addConstr(quicksum(X[i,j] for i in range(88))<=1)


# 6. Display cases next to cash registers
# only rtw and shoes
# Constraint for Belts (X65~X74): 10 units
# Constraint for Fashion Jewelry (X75 ~ X78): 4 units
# Constraint for SLG (X79~X88): 10 units
for i in belt:
    model.addConstr(quicksum(X[i, j] * Category[j][1] for j in range(num)) == 1)
    
for i in jewery:
    model.addConstr(quicksum(X[i, j] * Category[j][2] for j in range(num)) == 1)

for i in SLG:
    model.addConstr(quicksum(X[i, j] * Category[j][-1] for j in range(num)) == 1)
    
for i in range(24,64):
    model.addConstr(quicksum(X[i, j] * Category[j][5]+X[i, j] * Category[j][6] for j in range(num)) == 1)

for i in range(4,24):
    model.addConstr(quicksum(X[i, j] * Category[j][0]+X[i, j] * Category[j][3]+X[i, j] * Category[j][4] for j in range(num)) == 1)


model.addConstr(sum(X[0,j]*Category[j][0] for j in range(num)) == 1)
model.addConstr(sum(X[1,j]*Category[j][6] for j in range(num)) == 1)
model.addConstr(sum(X[2,j]*Category[j][0] for j in range(num)) == 1)
model.addConstr(sum(X[3,j]*Category[j][6] for j in range(num)) == 1)

model.addConstr(quicksum(X[i,j]*Gender[j][0] for i in women_clothings for j in range(num)) == 0)
model.addConstr(quicksum(X[i,j]*Gender[j][1] for i in men_clothings for j in range(num)) == 0)

birthseason = df_with_dummies["Birth Season"].apply(lambda x: 1 if x == "Spring / Summer 2019" else 0).tolist()
for i in shopwindow:
    model.addConstr(sum(X[i,j]*birthseason[j] for j in range(num)) == 1) 

# 7. The displayed items need to show at least 70% new seasonal products 
total_new_seasonal_items = quicksum(X[i, j] * birthseason[j] for i in range(88) for j in range(num))

model.addConstr(total_new_seasonal_items >= 0.7 * 88)


#12: Female Product Rate >= 50%
total_products = quicksum(quicksum(X[i, j] for j in range(num)) for i in range(88))
total_female_products = quicksum(quicksum(X[i, j] * Gender[j][1] for j in range(num)) for i in range(88))
model.addConstr(total_female_products >= 0.5 * total_products, "FemaleProductRate")


model.optimize()
for v in model.getVars():
    if v.x>0:
        print(f"{v.varName} = {round(v.x,2)}")
print("Sum of ne: ", (ne1.getValue() +ne2.getValue())/29)

print("Sum of vi1: ", vi1.getValue()/29)

print("Sum of rev: ", (rev1.getValue() + rev2.getValue())/29 )

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2407759
Academic license 2407759 - for non-commercial use only - registered to ti___@mail.mcgill.ca
Set parameter MIPGap to value 1e-12
location0CLUTCH VITELLO C/LAVORAZ. MEDUSA LASER-Black Gold Warm Gold = 1.0
location1SNEAKER FABRIC-White+Gold+Shell Pink = 1.0
location2BORSA NAPPA TRAP. + NAPPA TRAP. BAROCCO-Black+Multicolor-Tribute Gold = 1.0
location3SNEAKER FABRIC + CALF LEATHER-White+Gold = 1.0
location4BORSA  CALF-Black Warm Gold = 1.0
location5BORSA VITELLO + RICAMO LETTERING-BLACK-TRIBUTE GOLD = 1.0
location6BORSA VITELLO + ALTA FREQUENZA-BLACK-TRIBUTE GOLD = 1.0
location7BORSA NAPPA TRAPUNTATA-Off White-Tribute Gold = 1.0
location8BAG-BLACK = 1.0
location9CLUTCH VITELLO-Fuchsia-Tribute Gold = 1.0
location10ZAINO NYLON STAMPA "VERSACE"-Black+White = 1.0
location11BORSA NAPPA TRAPUNTATA-Off White-Black-Tribute Gold = 1.0
location12BAG-ROSSO = 1.0
location13BORSA CALF-Black Warm Gold = 1.0
locatio

## Problem Formulation - New Customer Priority (Base Model)

In [17]:
num = len(df_with_dummies)

model = gb.Model("Question1")
model.setParam('MIPGap', 0.000000000001)

# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

X = model.addVars(88,num,
                  vtype=GRB.BINARY, 
                  name=['location'+str(i)+j for i in range(88) for j in style])

Y = model.addVars(88,
                  vtype=GRB.BINARY, 
                  name=['location discount'+str(i) for i in range(88)])

ne1 = quicksum(new[j]*X[i,j] for i in range(4,88) for j in range(num))
ne2=quicksum(new[j]*X[i,j] for i in range(0,4) for j in range(num))*1.1

vi1 = quicksum(vip[j]*X[i,j] for i in range(88) for j in range(num))

rev1 = quicksum(revenue[j]*X[i,j] for i in range(4,88) for j in range(num))
rev2= quicksum(revenue[j]*X[i,j] for i in range(0,4) for j in range(num))*1.1


model.setObjectiveN(-(ne1+ne2), index=0, priority=2)
model.setObjectiveN(-rev1-rev2, index=1, priority=1)
model.setObjectiveN(-vi1, index=2, priority=0)

# 1 good for every postiion
for i in range(88):
    model.addConstr(quicksum(X[i,j] for j in range(num))==1)


#one product only appear once
for j in range(num):
    model.addConstr(quicksum(X[i,j] for i in range(88))<=1)


# 6. Display cases next to cash registers
# only rtw and shoes
# Constraint for Belts (X65~X74): 10 units
# Constraint for Fashion Jewelry (X75 ~ X78): 4 units
# Constraint for SLG (X79~X88): 10 units
for i in belt:
    model.addConstr(quicksum(X[i, j] * Category[j][1] for j in range(num)) == 1)
    
for i in jewery:
    model.addConstr(quicksum(X[i, j] * Category[j][2] for j in range(num)) == 1)

for i in SLG:
    model.addConstr(quicksum(X[i, j] * Category[j][-1] for j in range(num)) == 1)
    
for i in range(24,64):
    model.addConstr(quicksum(X[i, j] * Category[j][5]+X[i, j] * Category[j][6] for j in range(num)) == 1)

for i in range(4,24):
    model.addConstr(quicksum(X[i, j] * Category[j][0]+X[i, j] * Category[j][3]+X[i, j] * Category[j][4] for j in range(num)) == 1)


model.addConstr(sum(X[0,j]*Category[j][0] for j in range(num)) == 1)
model.addConstr(sum(X[1,j]*Category[j][6] for j in range(num)) == 1)
model.addConstr(sum(X[2,j]*Category[j][0] for j in range(num)) == 1)
model.addConstr(sum(X[3,j]*Category[j][6] for j in range(num)) == 1)

model.addConstr(quicksum(X[i,j]*Gender[j][0] for i in women_clothings for j in range(num)) == 0)
model.addConstr(quicksum(X[i,j]*Gender[j][1] for i in men_clothings for j in range(num)) == 0)

birthseason = df_with_dummies["Birth Season"].apply(lambda x: 1 if x == "Spring / Summer 2019" else 0).tolist()
for i in shopwindow:
    model.addConstr(sum(X[i,j]*birthseason[j] for j in range(num)) == 1) 

# 7. The displayed items need to show at least 70% new seasonal products 
total_new_seasonal_items = quicksum(X[i, j] * birthseason[j] for i in range(88) for j in range(num))

model.addConstr(total_new_seasonal_items >= 0.7 * 88)


#12: Female Product Rate >= 50%
total_products = quicksum(quicksum(X[i, j] for j in range(num)) for i in range(88))
total_female_products = quicksum(quicksum(X[i, j] * Gender[j][1] for j in range(num)) for i in range(88))
model.addConstr(total_female_products >= 0.5 * total_products, "FemaleProductRate")


model.optimize()
for v in model.getVars():
    if v.x>0:
        print(f"{v.varName} = {round(v.x,2)}")
print("Sum of ne: ", (ne1.getValue() +ne2.getValue())/29)

print("Sum of vi1: ", vi1.getValue()/29)

print("Sum of rev: ", (rev1.getValue() + rev2.getValue())/29 )

Set parameter MIPGap to value 1e-12
location0MARSUPIO NYLON STAMPA "VERSACE"-Black+White = 1.0
location1SNEAKER FABRIC-White+Gold+Shell Pink = 1.0
location2CLUTCH VITELLO C/LAVORAZ. MEDUSA LASER-Black Gold Warm Gold = 1.0
location3SNEAKER FABRIC + CALF LEATHER-White+Gold = 1.0
location4BORSA NAPPA TRAPUNTATA-Off White-Black-Tribute Gold = 1.0
location5CLUTCH CALF-Black Warm Gold = 1.0
location6BORSA VITELLO + RICAMO LETTERING-BLACK-TRIBUTE GOLD = 1.0
location7CLUTCH+CHAIN+METAL EYELET W/LION+STUDS GRAND CANYON-Optic White---Antique Gold = 1.0
location8BORSA VITELLO + ALTA FREQUENZA-Red-Tribute Gold = 1.0
location9BORSA VITELLO + ALTA FREQUENZA-Brilliant Blue-Tribute Gold = 1.0
location10BAG-BLACK = 1.0
location11BORSA NAPPA TRAP. + NAPPA TRAP. BAROCCO-Black+Multicolor-Tribute Gold = 1.0
location12CLUTCH+CHAIN+METAL EYELET W/LION GRAND CANYON-Black---Antique Gold = 1.0
location13CLUTCH+CHAIN+CRYSTAL EYELET W/LION GRAND CANYON-Black-Nickel = 1.0
location14EROS EAU DE TOILETTE NAT.SPRAY 1

## Problem Formulation - Revenue Priority

In [18]:
num = len(df_with_dummies)

model = gb.Model("Question1")
model.setParam('MIPGap', 0.000000000001)

# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

X = model.addVars(88,num,
                  vtype=GRB.BINARY, 
                  name=['location'+str(i)+j for i in range(88) for j in style])

Y = model.addVars(88,
                  vtype=GRB.BINARY, 
                  name=['location discount'+str(i) for i in range(88)])

ne1 = quicksum(new[j]*X[i,j] for i in range(4,88) for j in range(num))
ne2=quicksum(new[j]*X[i,j] for i in range(0,4) for j in range(num))*1.1

vi1 = quicksum(vip[j]*X[i,j] for i in range(88) for j in range(num))

rev1 = quicksum(revenue[j]*X[i,j] for i in range(4,88) for j in range(num))
rev2= quicksum(revenue[j]*X[i,j] for i in range(0,4) for j in range(num))*1.1


model.setObjectiveN(-(ne1+ne2), index=0, priority=1)
model.setObjectiveN(-rev1-rev2, index=1, priority=2)
model.setObjectiveN(-vi1, index=2, priority=0)

# 1 good for every postiion
for i in range(88):
    model.addConstr(quicksum(X[i,j] for j in range(num))==1)


#one product only appear once
for j in range(num):
    model.addConstr(quicksum(X[i,j] for i in range(88))<=1)


# 6. Display cases next to cash registers
# only rtw and shoes
# Constraint for Belts (X65~X74): 10 units
# Constraint for Fashion Jewelry (X75 ~ X78): 4 units
# Constraint for SLG (X79~X88): 10 units
for i in belt:
    model.addConstr(quicksum(X[i, j] * Category[j][1] for j in range(num)) == 1)
    
for i in jewery:
    model.addConstr(quicksum(X[i, j] * Category[j][2] for j in range(num)) == 1)

for i in SLG:
    model.addConstr(quicksum(X[i, j] * Category[j][-1] for j in range(num)) == 1)
    
for i in range(24,64):
    model.addConstr(quicksum(X[i, j] * Category[j][5]+X[i, j] * Category[j][6] for j in range(num)) == 1)

for i in range(4,24):
    model.addConstr(quicksum(X[i, j] * Category[j][0]+X[i, j] * Category[j][3]+X[i, j] * Category[j][4] for j in range(num)) == 1)


model.addConstr(sum(X[0,j]*Category[j][0] for j in range(num)) == 1)
model.addConstr(sum(X[1,j]*Category[j][6] for j in range(num)) == 1)
model.addConstr(sum(X[2,j]*Category[j][0] for j in range(num)) == 1)
model.addConstr(sum(X[3,j]*Category[j][6] for j in range(num)) == 1)

model.addConstr(quicksum(X[i,j]*Gender[j][0] for i in women_clothings for j in range(num)) == 0)
model.addConstr(quicksum(X[i,j]*Gender[j][1] for i in men_clothings for j in range(num)) == 0)

birthseason = df_with_dummies["Birth Season"].apply(lambda x: 1 if x == "Spring / Summer 2019" else 0).tolist()
for i in shopwindow:
    model.addConstr(sum(X[i,j]*birthseason[j] for j in range(num)) == 1) 

# 7. The displayed items need to show at least 70% new seasonal products 
total_new_seasonal_items = quicksum(X[i, j] * birthseason[j] for i in range(88) for j in range(num))

model.addConstr(total_new_seasonal_items >= 0.7 * 88)


#12: Female Product Rate >= 50%
total_products = quicksum(quicksum(X[i, j] for j in range(num)) for i in range(88))
total_female_products = quicksum(quicksum(X[i, j] * Gender[j][1] for j in range(num)) for i in range(88))
model.addConstr(total_female_products >= 0.5 * total_products, "FemaleProductRate")


model.optimize()
for v in model.getVars():
    if v.x>0:
        print(f"{v.varName} = {round(v.x,2)}")
print("Sum of ne: ", (ne1.getValue() +ne2.getValue())/29)

print("Sum of vi1: ", vi1.getValue()/29)

print("Sum of rev: ", (rev1.getValue() + rev2.getValue())/29 )

Set parameter MIPGap to value 1e-12
location0CLUTCH VITELLO C/LAVORAZ. MEDUSA LASER-Black Gold Warm Gold = 1.0
location1SNEAKER FABRIC-White+Gold+Shell Pink = 1.0
location2BORSA NAPPA TRAP. + NAPPA TRAP. BAROCCO-Black+Multicolor-Tribute Gold = 1.0
location3SNEAKER FABRIC + CALF LEATHER-White+Gold = 1.0
location4BORSA VITELLO + FOULARD BAROCCO-Black+Multicolor-Tribute Gold = 1.0
location5CLUTCH VITELLO-Fuchsia-Tribute Gold = 1.0
location6BORSA CALF-Black Warm Gold = 1.0
location7MARSUPIO NAPPA TRAPUNTATA-BLACK-TRIBUTE GOLD = 1.0
location8CLUTCH+CHAIN+CRYSTAL EYELET W/LION GRAND CANYON-Black-Nickel = 1.0
location9BORSA VITELLO + ALTA FREQUENZA-BLACK-TRIBUTE GOLD = 1.0
location10BORSA NAPPA TRAPUNTATA-Off White-Black-Tribute Gold = 1.0
location11BORSA VITELLO + RICAMO LETTERING-BLACK-TRIBUTE GOLD = 1.0
location12BORSA PITONE-Pink-Tribute Gold = 1.0
location13CLUTCH VITELLO ST.GRECA-Black-Palladium = 1.0
location14CLUTCH CALF-Black Warm Gold = 1.0
location15BORSA VITELLO + ALTA FREQUENZA-R

## Problem Extension - Including Discount

In [19]:
num = len(df_with_dummies)

model = gb.Model("Question1")
model.setParam('MIPGap', 0.000000000001)

# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

X = model.addVars(88,num,
                  vtype=GRB.BINARY, 
                  name=['location'+str(i)+j for i in range(88) for j in style])

Y = model.addVars(88,
                  vtype=GRB.BINARY, 
                  name=['location discount'+str(i) for i in range(88)])

Z = model.addVars(88, num, vtype=GRB.BINARY, name=['Z'+str(j)+str(i) for j in range(88) for i in range(num)])

ne1 = quicksum(new[j]*X[i,j] +Z[i,j]*new[j]*0.1 for i in range(4,88) for j in range(num))
ne2=quicksum(new[j]*X[i,j] for j in range(0,4) for j in range(num))*1.1

vi1 = quicksum(vip[j]*X[i,j] for i in range(88) for j in range(num))

rev1 = quicksum(revenue[j]*X[i,j]-0.065*Z[i,j]*revenue[j] for i in range(4,88) for j in range(num))
rev2= quicksum(revenue[j]*X[i,j] for i in range(0,4) for j in range(num))*1.1


model.setObjectiveN(-(ne1+ne2), index=0, priority=2)
model.setObjectiveN(-rev1-rev2, index=1, priority=1)
model.setObjectiveN(-vi1, index=2, priority=0)

# 1 good for every postiion
for i in range(88):
    model.addConstr(quicksum(X[i,j] for j in range(num))==1)


#one product only appear once max
for j in range(num):
    model.addConstr(quicksum(X[i,j] for i in range(88))<=1)

for i in range(88):
    for j in range(num):
        model.addConstr(Z[i,j] <= X[i,j], "Constr1_{}_{}".format(i,j))
        model.addConstr(Z[i,j] <= Y[i], "Constr2_{}_{}".format(i,j))
        model.addConstr(Z[i,j] >= X[i,j] + Y[i] - 1, "Constr3_{}_{}".format(i,j))


# 6. Display cases next to cash registers
# only rtw and shoes
# Constraint for Belts (X65~X74): 10 units
# Constraint for Fashion Jewelry (X75 ~ X78): 4 units
# Constraint for SLG (X79~X88): 10 units
for i in belt:
    model.addConstr(quicksum(X[i, j] * Category[j][1] for j in range(num)) == 1)
    
for i in jewery:
    model.addConstr(quicksum(X[i, j] * Category[j][2] for j in range(num)) == 1)

for i in SLG:
    model.addConstr(quicksum(X[i, j] * Category[j][-1] for j in range(num)) == 1)
    
for i in range(24,64):
    model.addConstr(quicksum(X[i, j] * Category[j][5]+X[i, j] * Category[j][6] for j in range(num)) == 1)

for i in range(4,24):
    model.addConstr(quicksum(X[i, j] * Category[j][0]+X[i, j] * Category[j][3]+X[i, j] * Category[j][4] for j in range(num)) == 1)


model.addConstr(sum(X[0,j]*Category[j][0] for j in range(num)) == 1)
model.addConstr(sum(X[1,j]*Category[j][6] for j in range(num)) == 1)
model.addConstr(sum(X[2,j]*Category[j][0] for j in range(num)) == 1)
model.addConstr(sum(X[3,j]*Category[j][6] for j in range(num)) == 1)

model.addConstr(quicksum(X[i,j]*Gender[j][0] for i in women_clothings for j in range(num)) == 0)
model.addConstr(quicksum(X[i,j]*Gender[j][1] for i in men_clothings for j in range(num)) == 0)

birthseason = df_with_dummies["Birth Season"].apply(lambda x: 1 if x == "Spring / Summer 2019" else 0).tolist()
for i in shopwindow:
    model.addConstr(sum(X[i,j]*birthseason[j] for j in range(num)) == 1) 

# 7. The displayed items need to show at least 70% new seasonal products 
total_new_seasonal_items = quicksum(X[i, j] * birthseason[j] for i in range(88) for j in range(num))

model.addConstr(total_new_seasonal_items >= 0.7* 88)

# 8. (Except for the new seasonal products) There are should be less than 15% of the discounted items
# Only items from seasons earlier than "Spring / Summer 2019" are discounted
total_discounted_items = quicksum(Y[i] for i in range(88))

model.addConstr(total_discounted_items <= 0.15 * 88)

#10: No Discount for Shopwindow
for i in shopwindow:
    for j in range(num):
        model.addConstr(X[i, j] + Y[i] <= 1, "NoDiscountInShopwindow_unit{}_product{}".format(i, j))
    
#11: No Discount for Birthseason Products
for i in range(88):
    for j in range(num):
        if birthseason[j] == 1:
            model.addConstr(X[i, j] +Y[i] <= 1, "NoDiscountForBirthseason_unit{}_product{}".format(i, j))
    
#12: Female Product Rate >= 50%
total_products = quicksum(quicksum(X[j, i] for i in range(num)) for j in range(88))
total_female_products = quicksum(quicksum(X[j, i] * Gender[i][1] for i in range(num)) for j in range(88))
model.addConstr(total_female_products >= 0.5 * total_products, "FemaleProductRate")

model.optimize()
for v in model.getVars():
    if v.x>0:
        print(f"{v.varName} = {round(v.x,2)}")
print("Sum of ne: ", (ne1.getValue() +ne2.getValue())/29)

print("Sum of vi1: ", vi1.getValue()/29)

print("Sum of rev: ", (rev1.getValue() + rev2.getValue())/29 )
for var in Y.values():
    print(var)

Set parameter MIPGap to value 1e-12
location0BORSA PITONE-Pink-Tribute Gold = 1.0
location1SNEAKER FABRIC-Sheer Lilac = 1.0
location2BORSA NAPPA TRAPUNTATA + PATCHWORK-White-Black-Multicolor-Tribute Gold = 1.0
location3SNEAKER FABRIC + CALF LEATHER-White+Gold = 1.0
location4EAU DE TOILETTE NAT. SPRAY 100 ML-NUL = 1.0
location5BAG-BLACK = 1.0
location6CLUTCH+CHAIN+METAL EYELET W/LION+STUDS GRAND CANYON-Optic White---Antique Gold = 1.0
location7EROS EAU DE TOILETTE NAT.SPRAY 100ML 100 ML SPRAY-NUL = 1.0
location8CLUTCH VITELLO C/LAVORAZ. MEDUSA LASER-Black Gold Warm Gold = 1.0
location9MARSUPIO NAPPA TRAPUNTATA-BLACK-TRIBUTE GOLD = 1.0
location10BORSA VITELLO + ALTA FREQUENZA-BLACK-TRIBUTE GOLD = 1.0
location11ZAINO NYLON STAMPA "VERSACE"-Black+White = 1.0
location12BORSA NAPPA TRAPUNTATA-Off White-Black-Tribute Gold = 1.0
location13BORSA NAPPA TRAP. + NAPPA TRAP. BAROCCO-Black+Multicolor-Tribute Gold = 1.0
location14BORSA VITELLO + FOULARD BAROCCO-Black+Multicolor-Tribute Gold = 1.0
loc